In [1]:
import dill as pickle
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
with open('trajectories_train.pkl', 'rb') as f:
    data = pickle.load(f)
with open('trajectories_test.pkl', 'rb') as f:
    data_test = pickle.load(f)

In [3]:
import pysindy as ps

def get_sindy_model(trajectories, actions,threshold=0.5,discrete_time=False,dt=2e-2):
    # optimizer = ps.STLSQ(threshold=threshold)
    optimizer = ps.SR3(threshold=threshold, thresholder='l1',trimming_fraction=0.2,max_iter=10000)
    lib = ps.GeneralizedLibrary([ps.PolynomialLibrary(), ps.FourierLibrary()])
    der = ps.SmoothedFiniteDifference()
    model = ps.SINDy(discrete_time=discrete_time, feature_library=lib, differentiation_method=der,
        optimizer=optimizer)
    model.fit(trajectories,u=actions,t=dt,multiple_trajectories=True)
    return model

In [4]:
def convert_data_to_traj_action(data):
    trajectories = [np.array(traj) for traj in data['states']]
    actions = [np.array(traj) for traj in data['actions']]
    return trajectories, actions

def convert_to_angle(trajs):
    trajs = [np.vstack((traj[:,0], np.arctan2(traj[:,1], traj[:,2]), traj[:,3], traj[:,4])).T for traj in trajs]
    return trajs

train_traj, train_actions = convert_data_to_traj_action(data)
test_traj, test_actions = convert_data_to_traj_action(data_test)
train_theta = convert_to_angle(train_traj)
test_theta = convert_to_angle(test_traj)

In [5]:
model_sincos = get_sindy_model(train_traj, train_actions, threshold=0.6)
print("model sin/cos train score:", model_sincos.score(train_traj,u=train_actions,t=0.02,multiple_trajectories=True))
print("model sin/cos test score:", model_sincos.score(test_traj,u=test_actions,t=0.02,multiple_trajectories=True))
model_sincos.print()

model sin/cos train score: 0.9216286421293697
model sin/cos test score: 0.9183733627306336
(x0)' = 0.994 x3
(x1)' = -0.993 x2 x4
(x2)' = 0.994 x1 x4
(x3)' = 13.805 u0 + 1.497 x1 x2 + 0.160 x1 x3 + 0.038 x2 u0 + -0.930 sin(1 x0) + 0.960 sin(1 x2) + -11.045 sin(1 u0)
(x4)' = -0.126 x3 + 0.579 u0 + -4.871 x0 u0 + -2.513 x1 x2 + 0.461 x1 x3 + -7.965 x1 u0 + 0.013 x2 u0 + -1.662 u0^2 + 0.284 sin(1 x1) + -23.299 sin(1 x2)


In [6]:
model = model_sincos
with open('sindy_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [7]:
model_theta = get_sindy_model(train_theta, train_actions, threshold=0.5)
print("model theta train score:", model_theta.score(train_theta,u=train_actions,t=0.02,multiple_trajectories=True))
print("model theta test score:", model_theta.score(test_theta,u=test_actions,t=0.02,multiple_trajectories=True))
model_theta.print()

model theta train score: 0.6265674906041794
model theta test score: 0.6210675414066351
(x0)' = 0.994 x2
(x1)' = 0.001 x3
(x2)' = 22.733 u0 + -0.137 x1 u0 + -0.327 u0^2 + 0.755 cos(1 x1) + -21.197 sin(1 u0) + -0.024 cos(1 u0)
(x3)' = 6.115 x0 + 6.367 x2 + -35.881 u0 + 1.233 x0 x1 + -6.791 x0 u0 + -2.150 x1 u0 + 14.564 x2^2 + -1.729 x2 u0 + -15.544 u0^2 + -6.550 sin(1 x0) + 0.529 cos(1 x0) + -20.023 cos(1 x1) + -7.290 sin(1 x2) + 32.729 cos(1 x2) + 40.301 sin(1 u0) + -33.039 cos(1 u0)
